In [1]:
from scipy import ndimage
import nibabel as nib
import random
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from torchvision.transforms import ToTensor

In [ ]:
sample_path = "C:/Users/DELL Inspiron/Desktop/ipnb projects/MosMedData/normal/study_0001.nii"


In [ ]:
img = read_nifty_file(sample_path)
normalized = normalize(img)
resized = resized_volume(normalized)

In [ ]:
normal_scan_paths = [os.path.join(os.getcwd(), "MosMedData/normal", x) for x in 
                    os.listdir("MosMedData/normal")]
abnormal_scan_paths = [
    os.path.join(os.getcwd(), "MosMedData/abnormal", x)
    for x in os.listdir("MosMedData/abnormal")
]


In [ ]:
class CTScanDataset(Dataset):
    def __init__(self, directory):
        super(CTScanDataset, self).__init__()
        self.files = glob.glob(directory+"/*/*")
        self.class_lookups = {"normal":0, "abnormal":1}
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file = self.files[idx]
        label = self.class_lookups[file.split(os.path.sep)[-2]]
        img = self.rotate(self.resize(self.normalize(self.read_nifti_file(file)))).unsqueeze(0)
        return img, label
    
    def read_nifti_file(self, file):
        scan = nib.load(file)
        scan = scan.get_fdata()
        return scan
    def normalize(self, img):
        max_value = np.max(img)
        min_value = np.min(img)
        normalized = (img-min_value)/(max_value-min_value)
        return normalized
    def resize(self, img):
        desired_h = 128
        desired_w = 128
        desired_d = 64
        
        h, w, d = img.shape
        resized = ndimage.zoom(img, (desired_h/h, desired_w/w, desired_d/d), order=1)
        return resized
    
    def rotate(self, volume):
        angles = [-20, -10, -5, 5, 10, 20]
        angle = random.choice(angles)
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume>1] = 1
        volume[volume<0] = 0
        return ToTensor()(volume)
        

In [ ]:
my_ds = CTScanDataset(directory="C:/Users/DELL Inspiron/Desktop/ipnb projects/MosMedData")

In [ ]:
my_ds[5][0].shape

In [ ]:
train_loader = DataLoader(my_ds, batch_size = 8, shuffle=True)

In [ ]:
# Hyperparameters
epochs = 5
learning_rate = 3e-4

In [7]:
class Classifier(nn.Module):
    def __init__(self, in_channels):
        super(Classifier, self).__init__()
        self.in_channels = in_channels
        self.block1 = nn.Sequential(nn.Conv3d(in_channels = self.in_channels,
                                        out_channels=256, kernel_size = 3, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool3d(kernel_size=2),
                                    nn.BatchNorm3d(256) # N x 256 x 32 x 64 x 64
                              )
        self.block2 = nn.Sequential(nn.Conv3d(in_channels=256, out_channels=128,
                                             kernel_size = 3, padding=1),
                                    nn.ReLU(),
                                   nn.MaxPool3d(kernel_size=2),
                                   nn.BatchNorm3d(128)) # N x 128 x 16 x 32 x 32
        
        self.block3 = nn.Sequential(nn.Conv3d(in_channels=128, out_channels=64,
                                             kernel_size=3, padding = 1),
                                    nn.ReLU(),
                                   nn.MaxPool3d(kernel_size=2),
                                   nn.BatchNorm3d(64))  # N x 64 x 8 x 16 x 16
        self.GlobalPool = nn.AdaptiveAvgPool3d(1)  # N x 64
        self.fc1 = nn.Sequential(nn.Linear(64, 512),
                                 nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(512, 1))
        
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x= self.block3(x)
        x = self.GlobalPool(x).squeeze()
        x = self.fc1(x)
        return x

In [8]:
model = Classifier(1)
x = torch.randn((2, 1, 64, 128, 128))

In [9]:
with torch.no_grad():
    test = model(x)

In [12]:
test.shape

torch.Size([2, 1])

In [ ]:
block1 = nn.Sequential(nn.Conv3d(in_channels = 1,
                                        out_channels=256, kernel_size = 3, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool3d(kernel_size=2),
                                    nn.BatchNorm3d(256) # N x 256 x 32 x 64 x 64
                              )
block2 = nn.Sequential(nn.Conv3d(in_channels=256, out_channels=128,
                                     kernel_size = 3, padding=1),
                            nn.ReLU(),
                           nn.MaxPool3d(kernel_size=2),
                           nn.BatchNorm3d(128)) # N x 128 x 16 x 32 x 32

block3 = nn.Sequential(nn.Conv3d(in_channels=128, out_channels=64,
                                     kernel_size=3, padding = 1),
                            nn.ReLU(),
                           nn.MaxPool3d(kernel_size=2),
                           nn.BatchNorm3d(64))  # N x 64 x 8 x 16 x 16
GlobalPool = nn.AdaptiveAvgPool3d(1)  # N x 64
fc1 = nn.Sequential(nn.Linear(64, 512),
                         nn.ReLU(),
                        nn.Dropout(0.3),
                        nn.Linear(512, 1))

In [2]:
x = torch.randn((2, 1, 64, 128, 128))

In [3]:
block11 = nn.Conv3d(1, 256, 3, padding=1)

In [4]:
test1 = block11(x)

In [ ]:
m = nn.Conv3d(1, 256, 3, padding=1)
input = torch.randn(2, 1, 64, 128, 128)
output = m(input)

In [ ]:
output.shape